In [1]:
!pip install langchain cohere python-dotenv gradio

In [2]:
!pip install langchain_community

In [3]:
!pip install cacheout

In [4]:
!pip install --upgrade langchain

In [5]:
import os
import gradio as gr
import cacheout
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.llms import Cohere
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.memory import ConversationBufferMemory

ImportError: cannot import name 'ToolsRenderer' from 'langchain_core.tools' (/usr/local/lib/python3.10/dist-packages/langchain_core/tools/__init__.py)

In [ ]:
os.environ['COHERE_API_KEY'] = "nhYrFwn0xNFthWrVy6YwUEGES2hMwsc0hRDjTnlh"
os.environ['TAVILY_API_KEY'] = "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"

In [ ]:
# # initialize cache
# cache = cacheout.Cache(maxsize=100, ttl=60)

In [ ]:
# Initialize API wrappers
search = TavilySearchResults()

In [ ]:
# Define tools
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )]

In [ ]:
# Initialize the Cohere language model
llm = Cohere(temperature=0.4, model="command-r-plus")
#chat = ChatCohere(model="command-r-plus", temperature=0.3)

In [ ]:
# Set up memory which allows for storing messages and then extracts the messages in a variable.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

Create the Chatbot Function

In [ ]:
def chatbot(message, history):
    # Convert history to the format expected by the agent
    chat_history = []
    for human, ai in history:
        chat_history.append(f"Human: {human}")
        chat_history.append(f"AI: {ai}")

    # Run the agent
    response = agent.run(input=message, chat_history=chat_history)

    # Return the response
    return response

In [ ]:
# cache = cacheout.Cache(maxsize=100, ttl=60)

In [ ]:
# def chatbot(message,history):
#   # create a cache key based on the input message and history
#   cache_key = f"{message}:{':'.join(f'{human}:{ai}' for human, ai in history)}"
#   # check if the result is cached
#   cached_response = cache.get(cache_key)
#   if cached_response:
#     return cached_response
#   # convert history to the format expected by the agent
#   chat_history = []
#   for human, ai in history:
#     chat_history.append(f"human: {human}")
#     chat_history.append(f"AI: {ai}")
#     # run the agent
#     response = agent.run(input=message, chat_history=chat_history)
#     # cache the result
#     cache.set(cache_key, response)
#     # return the response
#     return response

In [ ]:
# class chatbot:
#   def __init__(self,COHERE_API_KEY, cache_size=100,cache_ttl=60):
#     self.cohere_api_key = COHERE_API_KEY
#     self.cache = cacheout.Cache(maxsize=100, ttl=cache_ttl)
#     self.llm = Cohere(temperature=0.4, model="command-r-plus")
#   def get_response(self, prompt):
#     cached_response = self.cache.get(prompt)
#     if cached_response:
#       return cached_response
#     response = self.cohere_api_key.generate(prompt)
#     self.cache.set(prompt, response)
#   def clear_cache(self):
#     self.cache.clear()

Create the Gradio Interface

In [ ]:
iface = gr.ChatInterface(
    chatbot,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me anything...", container=False, scale=7),
    title="AI Agent",
    description="Ask me anything, and I'll use my tools to find the answer!",
    theme="soft",
    examples=[
        "What is the difference between generative and discriminative models?",
        "Describe the architecture of a Generative Adversarial Network and how the generator and discriminator interact during training.",
        "Explain the concept of a Variational Autoencoder (VAE) and how it incorporates latent variables into its architecture.",
        "How do conditional generative models differ from unconditional ones? Provide an example scenario where a conditional approach is beneficial.",
        "What is mode collapse in the context of GANs, and what strategies can be employed to address it during training?",
        "How does overfitting manifest in generative models, and what techniques can be used to prevent it during training?",
        "What is gradient clipping, and how does it help in stabilizing the training process of generative models?",
        "what is langchain?",
        "what is rnn? explain the architecture of rnn?",
        "Discuss strategies for training generative models when the available dataset is limited?",
        "Explain how curriculum learning can be applied in the training of generative models. What advantages does it offer?",
        "Describe the concept of learning rate scheduling and its role in optimizing the training process of generative models over time?"
    ],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
)

In [ ]:
# Launch the interface
iface.launch(debug=True)